In [7]:
##################################################
##################################################
######## IMPORT THE RELEVANT LIBRARIES
##################################################
##################################################
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import folium

from google.cloud import datastore
import googlemaps
from datetime import datetime

# setup a style to view ipython notebook graphs
sns.set_style('whitegrid')
sns.set_context('notebook')

%matplotlib inline
import datetime

import itertools

### library to get data from nyc_open_data
from sodapy import Socrata

In [8]:
####### grabbing the data regarding active dog licenses in 2016

client = Socrata("data.cityofnewyork.us", None)
results = client.get("tg4x-b46p", limit=50000)
film_permits = pd.DataFrame.from_records(results)

film_permits_since_2017 = film_permits[film_permits['startdatetime'] >= '2017-01-01']

In [9]:

API_KEY='_____'

In [10]:
#### setting up the geocode_result API

API_KEY = ''
gmaps = googlemaps.Client(key = API_KEY)

In [24]:
###https://gist.github.com/jlln/338b4b0b55bd6984f883
def splitDataFrameList(df,target_column,separator):
    ''' df = dataframe to split,
    target_column = the column containing the values to split
    separator = the symbol used to perform the split
    returns: a dataframe with each entry for the target column separated, with each element moved into a new row. 
    The values in the other columns are duplicated across the newly divided rows.
    '''
    def splitListToRows(row,row_accumulator,target_column,separator):
        split_row = row[target_column].split(separator)
        for s in split_row:
            new_row = row.to_dict()
            new_row[target_column] = s
            row_accumulator.append(new_row)
    new_rows = []
    df.apply(splitListToRows,axis=1,args = (new_rows,target_column,separator))
    new_df = pd.DataFrame(new_rows)
    return new_df

In [ ]:

# Instantiates a client
datastore_client = datastore.Client(project='np-training')
gmaps = googlemaps.Client(key=API_KEY)
 
def geocode(location, kind='Location'):
    task_key = datastore_client.key(kind, location)
 
    # Prepares the new entity
    task = datastore_client.get(task_key)
    if task is None :
        task = datastore.Entity(task_key)
       
    payload = task.get('payload',None)
   
    if "geocodable" not in task:
        if "payload" not in task:
            payload = gmaps.geocode(location)
            task['payload'] = payload
            if len(payload) > 0:
                geometry = payload[0]['geometry']
                task['geometry']= geometry
                task['location'] =geometry['location']
                task['geocodable'] = True
            else:
                task['geocodable'] = False
               
        else:
            task['geocodable'] = True
        datastore_client.put(task)
       
       
 
    return payload
    #return task["location"]
   
 
raw_address_mapping = {}
 
# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    # Start the load operations and mark each future with its URL
    future_to_geocode = {executor.submit(geocode, location): location for location in all_locations}
    for future in tqdm(concurrent.futures.as_completed(future_to_geocode)):
        location = future_to_geocode[future]
        try:
            data = future.result()
            raw_address_mapping[location] = data
            #geocoded_list.append({"location":location, "lat" :data["lat"],  "long": data["long"]})
        except Exception as exc:
            print (exc)
            print (f"Failed for {location}")
 
 
df.apply(lambda x: pd.Series(x['locations']),axis=1).stack().reset_index(level=1, drop=True)
 
s = df.apply(lambda x: pd.Series(x['locations']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'location'
df = df.join(s)


In [5]:
###############################
### Data exploration
###############################
##film_permits_since_2017['subcategoryname'].value_counts()

In [25]:
splitDataFrameList(film_permits_since_2017, 'parkingheld', ',')

,borough,category,communityboard_s,country,enddatetime,enteredon,eventagency,eventid,eventtype,parkingheld,policeprecinct_s,startdatetime,subcategoryname,zipcode_s
0,Manhattan,Television,5,United States of America,2017-01-02T09:00:00,2016-12-30T05:47:38,"Mayor's Office of Film, Theatre & Broadcasting",325488,Shooting Permit,WEST 44 STREET between BROADWAY and 6 AVENUE,14,2017-01-02T00:00:00,Morning Show,10036
1,Bronx,Television,1,United States of America,2017-01-03T23:00:00,2016-12-16T16:55:04,"Mayor's Office of Film, Theatre & Broadcasting",324505,Shooting Permit,LOCUST AVENUE between EAST 139 STREET and EAS...,40,2017-01-03T07:00:00,Episodic series,10454
2,Bronx,Television,1,United States of America,2017-01-03T23:00:00,2016-12-16T16:55:04,"Mayor's Office of Film, Theatre & Broadcasting",324505,Shooting Permit,EAST 140 STREET between LOCUST AVENUE and W...,40,2017-01-03T07:00:00,Episodic series,10454
3,Bronx,Television,1,United States of America,2017-01-03T23:00:00,2016-12-16T16:55:04,"Mayor's Office of Film, Theatre & Broadcasting",324505,Shooting Permit,WALNUT AVENUE between EAST 139 STREET and E...,40,2017-01-03T07:00:00,Episodic series,10454
4,Bronx,Television,1,United States of America,2017-01-03T23:00:00,2016-12-16T16:55:04,"Mayor's Office of Film, Theatre & Broadcasting",324505,Shooting Permit,EAST 139 STREET between WALNUT AVENUE and L...,40,2017-01-03T07:00:00,Episodic series,10454
5,Brooklyn,Television,6,United States of America,2017-01-03T23:59:00,2016-12-16T11:39:36,"Mayor's Office of Film, Theatre & Broadcasting",324416,Shooting Permit,NEVINS STREET between UNION STREET and DEGRAW ...,78,2017-01-03T06:00:00,Cable-episodic,11217
6,Brooklyn,Television,6,United States of America,2017-01-03T23:59:00,2016-12-16T11:39:36,"Mayor's Office of Film, Theatre & Broadcasting",324416,Shooting Permit,DEGRAW STREET between 3 AVENUE and GOWANUS C...,78,2017-01-03T06:00:00,Cable-episodic,11217
7,Brooklyn,Television,6,United States of America,2017-01-03T23:59:00,2016-12-16T11:39:36,"Mayor's Office of Film, Theatre & Broadcasting",324416,Shooting Permit,DOUGLASS STREET between 3 AVENUE and NEVINS ...,78,2017-01-03T06:00:00,Cable-episodic,11217
8,Brooklyn,Television,6,United States of America,2017-01-03T23:59:00,2016-12-16T11:39:36,"Mayor's Office of Film, Theatre & Broadcasting",324416,Shooting Permit,SACKETT STREET between 3 AVENUE and NEVINS S...,78,2017-01-03T06:00:00,Cable-episodic,11217
9,Manhattan,Theater,5,United States of America,2017-01-05T01:00:00,2016-12-15T11:16:26,"Mayor's Office of Film, Theatre & Broadcasting",324242,Shooting Permit,WEST 48 STREET between 8 AVENUE and BROADWAY,18,2017-01-02T06:00:00,Theater,"10019, 10036"


In [10]:
#####################################################
########## Turn the parkingheld variable into a list 
#####################################################

def get_locations_with_borough(row):
    locations = row['parkingheld']
    borough = row['borough']
    
    parsed_locations = locations.split(",")  
    parsed_locations = [ l.strip() + ", " + borough for l in parsed_locations ] 
    return pd.Series([parsed_locations])

film_permits_since_2017['locations'] = film_permits_since_2017.apply(get_locations_with_borough, axis = 1)

/Users/talmadge.farringer/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [9]:
##film_permits_since_2017[film_permits_since_2017['subcategoryname'] == 'Signed Artist']

In [12]:
from google.cloud import datastore
import googlemaps

# Instantiates a client
datastore_client = datastore.Client(project='np-training')
gmaps = googlemaps.Client(key=API_KEY)

def geocode(location, kind='Location'):
    task_key = datastore_client.key(kind, location)

    # Prepares the new entity
    task = datastore.Entity(key=task_key)
    payload = None
    if task is None or payload not in task:
        payload = gmaps.geocode(location)   
        task['payload'] = payload
        if len(payload) > 0:
            geometry = payload[0]['geometry']
            task['geometry']= geometry
            task['location'] =geometry['location']
        datastore_client.put(task)
    else:
        payload = task['payload']
        
    #return payload
    return payload

In [195]:
#### find all of the distinct locations
distinct_locations = sorted(list(set(list(itertools.chain(*film_permits_since_2017['locations'])))))

##### how many distinct locations are there?
len(distinct_locations)


12799

In [159]:
some_locations = distinct_locations[:10]

In [160]:
raw_address_mapping = {}

In [161]:
####### Run the distinct locations through the google geocode api
for location in some_locations:
    payload = geocode_result = gmaps.geocode(location)
    raw_address_mapping[location] = payload

In [162]:
####### Combine the results into a dataframe
keys = list(raw_address_mapping.keys())
values = list(raw_address_mapping.values())
mapping_df = pd.DataFrame({'location':keys, 'payload':values})

mapping_df['location_coordinates']=mapping_df['payload'].apply(lambda x: x[0]['geometry']['location'])
### create column for longitude and latitude 
mapping_df['lat'] = mapping_df['location_coordinates'].apply(lambda x:x['lat'])
mapping_df['long'] = mapping_df['location_coordinates'].apply(lambda x: x['lng'])



In [186]:
from folium.plugins import MarkerCluster
starting_location = [mapping_df.iloc[0]['lat'],mapping_df.iloc[0]['long']]

colors = {'A' : 'red', 'B' : 'blue'}

map_osm = folium.Map(location=starting_location, zoom_start=10,tiles='cartodbpositron')

mc = MarkerCluster()



#creating a Marker for each point in df_sample. Each point will get a popup with their zip
for row in mapping_df.itertuples():
    mc.add_child(folium.CircleMarker(location=[row.lat,  row.long]
                                             , radius=5
                                             , color='#3186cc'
                                             , fill_color='#3186cc'
                                             , fill = True
                                             , popup=row.location))
map_osm.add_child(mc)



map_osm